# Part 3 - Model Tuning and Evaluation

In [1]:
# import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso,LassoCV,Ridge,RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline

In [2]:
#import data
housing_df=pd.read_csv("../datasets/modelling_housing_data.csv")
housing_df.head()

C:\Users\songk\AppData\Local\Temp\ipykernel_23840\3244853293.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_df=pd.read_csv("../datasets/modelling_housing_data.csv")


,id,tranc_yearmonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,sec_sch_affiliation,sec_sch_latitude,sec_sch_longitude
0,88471,2016-05,KALLANG/WHAMPOA,4 ROOM,3B,UPP BOON KENG RD,10 TO 12,90.0,Model A,2006,...,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504
1,122598,2012-07,BISHAN,5 ROOM,153,BISHAN ST 13,07 TO 09,130.0,Improved,1987,...,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892
2,170897,2013-07,BUKIT BATOK,EXECUTIVE,289B,BT BATOK ST 25,13 TO 15,144.0,Apartment,1997,...,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013
3,86070,2012-04,BISHAN,4 ROOM,232,BISHAN ST 22,01 TO 05,103.0,Model A,1992,...,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934
4,153632,2017-12,YISHUN,4 ROOM,876,YISHUN ST 81,01 TO 03,83.0,Simplified,1987,...,74,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,0,1.414888,103.838335


In [3]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150634 entries, 0 to 150633
Data columns (total 77 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         150634 non-null  int64  
 1   tranc_yearmonth            150634 non-null  object 
 2   town                       150634 non-null  object 
 3   flat_type                  150634 non-null  object 
 4   block                      150634 non-null  object 
 5   street_name                150634 non-null  object 
 6   storey_range               150634 non-null  object 
 7   floor_area_sqm             150634 non-null  float64
 8   flat_model                 150634 non-null  object 
 9   lease_commence_date        150634 non-null  int64  
 10  resale_price               150634 non-null  float64
 11  tranc_year                 150634 non-null  int64  
 12  tranc_month                150634 non-null  int64  
 13  mid_storey                 15

In [4]:
housing_df['id']=housing_df['id'].map(lambda x: str(x))

In [5]:
housing_df.describe()

,floor_area_sqm,lease_commence_date,resale_price,tranc_year,tranc_month,mid_storey,lower,upper,mid,floor_area_sqft,...,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,cutoff_point,sec_sch_affiliation,sec_sch_latitude,sec_sch_longitude
count,150634.00000,150634.000000,1.506340e+05,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,...,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000
mean,97.19763,1992.448464,4.491615e+05,2016.489551,6.580500,8.284823,7.248370,9.321275,8.284823,1046.235290,...,395.172974,55.368190,0.113195,1.366268,103.839106,508.533220,210.146428,0.031879,1.365986,103.839340
std,24.40304,12.138829,1.433076e+05,2.752396,3.345468,5.508074,5.515155,5.507364,5.508074,262.674321,...,234.760931,17.903918,0.316832,0.042669,0.072637,309.066265,20.010489,0.175677,0.042599,0.072630
min,31.00000,1966.000000,1.500000e+05,2012.000000,1.000000,2.000000,1.000000,3.000000,2.000000,333.684000,...,45.668324,20.000000,0.000000,1.274962,103.687724,38.913475,188.000000,0.000000,1.276029,103.687207
25%,75.00000,1984.000000,3.470000e+05,2014.000000,4.000000,5.000000,4.000000,6.000000,5.000000,807.300000,...,227.083164,44.000000,0.000000,1.337289,103.773754,290.285883,188.000000,0.000000,1.337545,103.776008
50%,95.00000,1992.000000,4.200000e+05,2017.000000,7.000000,8.000000,7.000000,9.000000,8.000000,1022.580000,...,348.876691,54.000000,0.000000,1.364639,103.844210,447.377670,208.000000,0.000000,1.366132,103.842719
75%,112.00000,2001.000000,5.200000e+05,2019.000000,9.000000,11.000000,10.000000,12.000000,11.000000,1205.568000,...,502.179385,67.000000,0.000000,1.393910,103.898773,644.284099,224.000000,0.000000,1.391781,103.899872
max,280.00000,2019.000000,1.258000e+06,2021.000000,12.000000,50.000000,49.000000,51.000000,50.000000,3013.920000,...,3305.841039,110.000000,1.000000,1.456667,103.962919,3638.977233,260.000000,1.000000,1.453280,103.961105


### Feature Selection

The following features will be selected for modelling resale price based on our EDA, as summarised in below table.


|   Variable Type   | Model Feature |
| ----------------------| ------------- |
| Numerical | hdb_age |
| Numerical | mid |
| Numerical | max_floor_lvl  |
| Numerical | floor_area_sqft |
| Numerical | bus_interchange |
| Numerical | mrt_interchange |
| Numerical |mall_nearest_distance|
| Numerical |mrt_nearest_distance| 
| Numerical |hawker_nearest_distance| 
| Numerical |pri_sch_nearest_distance| 
| Numerical |sec_sch_nearest_dist| 
| Categorical | planning_area|
| Categorical | full_flat_type|

As planning area and full flat type are categorical variables, one-hot encoding will be applied to convert them into numerical variables for modelling purposes.

### Baseline Model

We establish the baseline model using a dummy regressor that predicts the resale price as the median resale price of the training dataset.

In [6]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

In [7]:
X.head()

,hdb_age,mid,max_floor_lvl,floor_area_sqft,pri_sch_nearest_distance,sec_sch_nearest_dist,hawker_nearest_distance,mrt_nearest_distance,bus_interchange,mrt_interchange,...,full_flat_type_5 ROOM Premium Apartment,full_flat_type_5 ROOM Premium Apartment Loft,full_flat_type_5 ROOM Standard,full_flat_type_5 ROOM Type S2,full_flat_type_EXECUTIVE Adjoined flat,full_flat_type_EXECUTIVE Apartment,full_flat_type_EXECUTIVE Maisonette,full_flat_type_EXECUTIVE Premium Apartment,full_flat_type_EXECUTIVE Premium Maisonette,full_flat_type_MULTI-GENERATION Multi Generation
0,15,11,25,968.760,1138.633422,1138.633422,154.753357,330.083069,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34,8,9,1399.320,415.607357,447.894399,640.151925,903.659703,1,1,...,0,0,0,0,0,0,0,0,0,0
2,24,14,16,1550.016,498.849039,180.074558,1762.082341,1334.251197,1,0,...,0,0,0,0,0,1,0,0,0,0
3,29,3,11,1108.692,389.515528,389.515528,726.215262,907.453484,1,1,...,0,0,0,0,0,0,0,0,0,0
4,34,2,4,893.412,401.200584,312.025435,1540.151439,412.343032,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=30)
base_model = DummyRegressor(strategy='median')
base_model.fit(X_train, y_train)
y_pred=base_model.predict(X_test)
y_pred_train=base_model.predict(X_train)
baseline_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred))
baseline_cross_val_score=cross_val_score(base_model,X_train,y_train,cv=5).mean()
baseline_train_R2_score=r2_score(y_train,y_pred_train)
baseline_test_R2_score=r2_score(y_test,y_pred)

print(f"Baseline RMSE: {baseline_RMSE}")
print(f"Baseline 5-fold validation score: {baseline_cross_val_score}")
print(f"Train R2 score: {baseline_train_R2_score}")
print(f"Test R2 score: {baseline_test_R2_score}")

Baseline RMSE: 145936.9909607785
Baseline 5-fold validation score: -0.04126297641121126
Train R2 score: -0.041743740890128445
Test R2 score: -0.04040739192181375


### Linear Regression

In [9]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

In [10]:
#linear regression
lr = LinearRegression()
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
lr_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred))
lr_cross_val_score=cross_val_score(lr,X_train,y_train,cv=5).mean()
lr_train_R2_score=r2_score(lr.predict(X_train), y_train)
lr_test_R2_score=r2_score(lr.predict(X_test), y_test)
print(f"Linear regression RMSE: {lr_RMSE}")
print(f"Linear regression 5-fold validation score: {lr_cross_val_score}")
print(f"Linear regression train R2 score: {lr_train_R2_score}")
print(f"Linear regression test R2 score: {lr_test_R2_score}")

Linear regression RMSE: 51134.513808055555
Linear regression 5-fold validation score: 0.872533785785269
Linear regression train R2 score: 0.8543199882273074
Linear regression test R2 score: 0.8537299864699261


In [11]:
lr.intercept_

206971.449092748

In [12]:
lr.coef_

array([-3.43718138e+03,  3.35804301e+03,  2.79838494e+03,  3.08446031e+02,
        1.05907835e+01,  9.70629399e+00, -1.81052851e+01, -5.82590478e+01,
        6.90462000e+03,  4.09951616e+03, -1.62241586e+01, -1.30545667e+04,
        9.49794095e+04, -4.20227184e+04,  8.61643857e+04, -1.09099527e+05,
        1.55308162e+05,  1.61491076e+05, -1.05278927e+05,  2.75506805e+04,
        1.63828895e+05,  2.11031361e+04, -4.87468407e+04, -4.94378069e+04,
       -9.04421858e+04,  3.24953610e+04,  2.02978044e+05,  2.32592536e+04,
        7.78551769e+04, -5.99645438e+04, -3.52638604e+04,  7.85397257e+04,
        1.47846866e+04, -1.42759843e+05, -7.34778435e+04,  3.51139720e+04,
       -2.59442886e+04,  1.95739119e+05,  3.36813654e+04, -7.60097035e+04,
       -1.34796466e+05, -7.78352382e+04,  1.49687389e+04, -1.25388074e+04,
       -6.69079255e+03, -4.55309990e+03,  1.52479346e+04, -1.70043368e+04,
        6.28354170e+04,  7.51902186e+03,  3.18894905e+04,  3.04606663e+04,
        1.66441337e+04,  

### Lasso Regression

In [14]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)

In [15]:
alphas = np.logspace(-4, 4, 2000)
LS = LassoCV(alphas = alphas ,cv=5, random_state=30).fit(X_train, y_train)
y_pred_with_lasso = LS.predict(X_test)

lasso_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred_with_lasso))
lasso_train_R2_score=r2_score(LS.predict(X_train), y_train)
lasso_test_R2_score=r2_score(LS.predict(X_test), y_test)
print(f"alpha: {LS.alpha_}")
print(f"Lasso RMSE: {lasso_RMSE}")
print(f"Lasso regression train R2 score: {lasso_train_R2_score}")
print(f"Lasso regression test R2 score: {lasso_test_R2_score}")

C:\Users\songk\anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9645002505754.188, tolerance: 186180757793.9913
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\songk\anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15168200466542.219, tolerance: 186180757793.9913
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\songk\anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 422200691229.9375, tolerance: 186180757793.9913
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\songk\anaconda3\envs\dsi-sg\lib\site-packages\sk

alpha: 0.02637610149518831
Lasso RMSE: 51141.692080904635
Lasso regression train R2 score: 0.8543012736081093
Lasso regression test R2 score: 0.853683651823702


C:\Users\songk\anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.477e+14, tolerance: 2.323e+11
  model = cd_fast.enet_coordinate_descent(


In [16]:
LS.coef_

array([-3.45052771e+03,  3.35797829e+03,  2.80615721e+03,  3.11151433e+02,
        1.05709633e+01,  9.75493227e+00, -1.81118550e+01, -5.83327643e+01,
        6.90131551e+03,  4.10095315e+03, -1.62600565e+01, -1.30598813e+04,
        9.49149990e+04, -4.20233852e+04,  8.57983026e+04, -1.09106147e+05,
        1.55190402e+05,  1.61471503e+05, -1.05343319e+05,  2.75578257e+04,
        1.63652397e+05,  2.10807321e+04, -4.87582303e+04, -4.94709484e+04,
       -9.04082099e+04,  3.24010548e+04,  2.02938538e+05,  2.32616227e+04,
        7.76948948e+04, -6.00749083e+04, -3.52143257e+04,  7.84307263e+04,
        1.46907893e+04, -1.42693379e+05, -7.34222717e+04,  3.51081399e+04,
       -2.59576063e+04,  1.95380011e+05,  3.36200287e+04, -7.59325585e+04,
       -1.34888847e+05, -7.78306523e+04, -5.36757595e+03, -3.07901092e+04,
       -2.73093930e+04, -2.57107482e+04, -5.94146101e+03, -3.74227790e+04,
        4.09959281e+04, -1.36401141e+04,  1.02500695e+04,  9.07255610e+03,
       -5.17686419e+03,  

In [17]:
LS.intercept_

226837.7622567633

### Ridge Regression

In [18]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)

In [19]:
alphas = np.logspace(-4, 4, 2000)
ridge = RidgeCV(alphas = alphas ,cv=5).fit(X_train, y_train)
y_pred_with_ridge = ridge.predict(X_test)

ridge_RMSE=np.sqrt(metrics.mean_squared_error(y_test,y_pred_with_ridge))
ridge_train_R2_score=r2_score(ridge.predict(X_train), y_train)
ridge_test_R2_score=r2_score(ridge.predict(X_test), y_test)
print(f"Ridge regression alpha: {ridge.alpha_}")
print(f"Ridge regression RMSE: {ridge_RMSE}")
print(f"Ridge regression train R2 score: {ridge_train_R2_score}")
print(f"Ridge regression test R2 score: {ridge_test_R2_score}")

Ridge regression alpha: 0.014095306931660116
Ridge regression RMSE: 51134.64529377879
Ridge regression train R2 score: 0.8543175906531845
Ridge regression test R2 score: 0.8537270840438869


In [20]:
ridge.coef_

array([-3.43808711e+03,  3.35805409e+03,  2.79888522e+03,  3.08647872e+02,
        1.05898861e+01,  9.70968045e+00, -1.81066914e+01, -5.82632774e+01,
        6.90419123e+03,  4.09937376e+03, -1.62246964e+01, -1.30560815e+04,
        9.49740117e+04, -4.20223331e+04,  8.61445289e+04, -1.09098727e+05,
        1.55291841e+05,  1.61380717e+05, -1.05280106e+05,  2.75510718e+04,
        1.63741364e+05,  2.11017524e+04, -4.87469127e+04, -4.94413932e+04,
       -9.04391342e+04,  3.24886453e+04,  2.02969237e+05,  2.32646147e+04,
        7.78579429e+04, -5.99724894e+04, -3.52551838e+04,  7.85359807e+04,
        1.47792714e+04, -1.42751873e+05, -7.34710685e+04,  3.51131582e+04,
       -2.59453524e+04,  1.95561027e+05,  3.36768587e+04, -7.59347385e+04,
       -1.34802841e+05, -7.78348673e+04,  1.38589932e+04, -1.34200597e+04,
       -7.72179182e+03, -5.61982415e+03,  1.41650186e+04, -1.80220124e+04,
        6.17153563e+04,  6.44852230e+03,  3.07855914e+04,  2.93743477e+04,
        1.55265200e+04,  

In [21]:
ridge.intercept_

207940.6081465045

No significant improvement was noted in terms of RMSE or R2 score when Lasso or Ridge regularisation was applied. We will rely on existing linear regression model to further understand our coefficients, since there was a significant improvement observed relative to the baseline score.

In [22]:
X = housing_df[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)
y = housing_df['resale_price']

In [23]:
lr = LinearRegression()
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=30)
lr.fit(X_train,y_train)

LinearRegression()

In [24]:
coefficients={'Feature':[x for x in X_train.columns],
              'Coefficient Value':[x for x in lr.coef_]}

coefficients['Category'] = ['full_flat_type' if 'full_flat_type' in var 
                          else 'planning_area' if 'planning_area' in var
                          else 'mrt/bus' if ('mrt' in var) | ('bus' in var)
                          else 'hawker' if 'hawker' in var
                          else 'storey' if (var == 'mid') | ('storey' in var) | ('lvl' in var)
                          else 'mall' if 'mall' in var
                          else 'time' if 'tranc' in var
                          else 'sch' if 'sch' in var
                          else var for var in X_train.columns]

In [25]:
coeff_table=pd.DataFrame(coefficients)
coeff_table.to_csv("../datasets/model_coefficients.csv",index=False)